In [ ]:
import boto3
import sagemaker
from sagemaker import AutoML

# Initialize SageMaker session
session = sagemaker.Session()
role = "arn:aws:iam::940482431470:role/service-role/AmazonSageMaker-ExecutionRole-20241118T144464"

# Define the input and output S3 paths
s3_input_path = "s3://machine-failure-transformed-bucket/TransformedData/part-00000-9620d974-1176-4141-9a5b-93f56d422c48-c000.csv"
s3_output_path = "s3://machine-failure-transformed-bucket/AutopilotOutput/"

# Initialize the AutoML job
automl = AutoML(
    role=role,
    target_attribute_name="Failure",  
    output_path=s3_output_path,
    problem_type="BinaryClassification", 
    job_objective={"MetricName": "Accuracy"},
)
# Launch the AutoML job
automl_job_name = "failure-prediction-automl"
automl.fit(inputs=s3_input_path, job_name=automl_job_name)

print("AutoML job launched:", automl_job_name)

..........................................................................................................................................................

KeyboardInterrupt: 

In [28]:
import boto3
import sagemaker
from sagemaker import AutoML
automl_job_name = "failure-prediction-automl"
client = boto3.client("sagemaker")
response = client.describe_auto_ml_job(AutoMLJobName=automl_job_name)
print("AutoML Job Status:", response["AutoMLJobStatus"])

AutoML Job Status: Completed


In [29]:
response = client.describe_auto_ml_job(AutoMLJobName=automl_job_name)

# Print the overall job status
print("AutoML Job Status:", response["AutoMLJobStatus"])

# Retrieve and display the best candidate
best_candidate = response["BestCandidate"]
print("Best Candidate Details:")
print("Candidate Name:", best_candidate["CandidateName"])
print("Objective Metric:", best_candidate["FinalAutoMLJobObjectiveMetric"])

# Retrieve model inference details
inference_containers = best_candidate["InferenceContainers"]
print("Model Details:")
for container in inference_containers:
    print("Container Image:", container["Image"])
    if "ModelPackageName" in container:
        print("Model Package Name:", container["ModelPackageName"])
    if "Environment" in container:
        print("Environment Variables:", container["Environment"])

AutoML Job Status: Completed
Best Candidate Details:
Candidate Name: failure-prediction-automl5JDFhCR-010-07b5b725
Objective Metric: {'MetricName': 'validation:accuracy', 'Value': 0.9915000200271606, 'StandardMetricName': 'Accuracy'}
Model Details:
Container Image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.5-1-cpu-py3
Environment Variables: {'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
Container Image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3
Environment Variables: {'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,probabilities'}
Container Image: 683313688378.dkr.e

In [30]:
response = client.list_candidates_for_auto_ml_job(AutoMLJobName=automl_job_name)

# Print details of each candidate
print("All Models and Their Details:")
for candidate in response["Candidates"]:
    print("\nCandidate Name:", candidate["CandidateName"])
    print("Status:", candidate["CandidateStatus"])
    print("Objective Metric:", candidate["FinalAutoMLJobObjectiveMetric"])
    print("Metrics:")
    for metric in candidate.get("CandidateProperties", {}).get("CandidateMetrics", []):
        print(f" - {metric['MetricName']}: {metric['Value']} (Set: {metric['Set']})")
    print("Inference Containers:")
    for container in candidate.get("InferenceContainers", []):
        print(" - Container Image:", container["Image"])
        print(" - Environment Variables:", container.get("Environment", {}))

All Models and Their Details:

Candidate Name: failure-prediction-automl5JDFhCR-100-fe38ede6
Status: Completed
Objective Metric: {'MetricName': 'validation:accuracy', 'Value': 0.90967857837677, 'StandardMetricName': 'Accuracy'}
Metrics:
 - LogLoss: 0.2515617907047272 (Set: Validation)
 - Accuracy: 0.90967857837677 (Set: Validation)
 - Recall: 0.9086808562278748 (Set: Validation)
 - Precision: 0.2686040699481964 (Set: Validation)
 - AUC: 0.9605914354324341 (Set: Validation)
 - F1: 0.4139392375946045 (Set: Validation)
 - BalancedAccuracy: 0.9091978073120117 (Set: Validation)
Inference Containers:
 - Container Image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.5-1-cpu-py3
 - Environment Variables: {'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
 - Container Image: 382416733822.dkr.ecr.us-east-1.ama

In [46]:
print(best_candidate["CandidateProperties"]["CandidateArtifactLocations"])

{'Explainability': 's3://machine-failure-transformed-bucket/AutopilotOutput/failure-prediction-automl/documentation/explainability/output', 'ModelInsights': 's3://machine-failure-transformed-bucket/AutopilotOutput/failure-prediction-automl/documentation/model_monitor/output'}


In [47]:
print(best_candidate["InferenceContainers"])

[{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.5-1-cpu-py3', 'ModelDataUrl': 's3://machine-failure-transformed-bucket/AutopilotOutput/failure-prediction-automl/data-processor-models/failure-prediction-automl-dpp5-1-d5bde47af9624719a567278df21a3b/output/model.tar.gz', 'Environment': {'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}}, {'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3', 'ModelDataUrl': 's3://machine-failure-transformed-bucket/AutopilotOutput/failure-prediction-automl/tuning/failure-pr-dpp5-xgb/failure-prediction-automl5JDFhCR-010-07b5b725/output/model.tar.gz', 'Environment': {'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': '

In [49]:
xgboost_container = {
    "Image": "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3",
    "ModelDataUrl": "s3://machine-failure-transformed-bucket/AutopilotOutput/failure-prediction-automl/tuning/failure-pr-dpp5-xgb/failure-prediction-automl5JDFhCR-010-07b5b725/output/model.tar.gz",
}

from sagemaker.model import Model

xgboost_model = Model(
    model_data=xgboost_container["ModelDataUrl"],
    image_uri=xgboost_container["Image"],
    role=role,
    sagemaker_session=session
)

endpoint_name = "failure-prediction-xgboost-endpoint"

predictor = xgboost_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name
)

print(f"XGBoost endpoint deployed: {endpoint_name}")


-------!XGBoost endpoint deployed: failure-prediction-xgboost-endpoint


In [50]:

import boto3
import json

# Initialize SageMaker runtime client
runtime_client = boto3.client("sagemaker-runtime")

# Define your endpoint name
endpoint_name = "failure-prediction-xgboost-endpoint"

# Example payload: replace with your feature data
test_payload = "28.1,38.6,11,4.8,0,1.5,61.584688"  

# Send the request
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="text/csv",  
    Body=test_payload
)

# Parse the response
result = response["Body"].read().decode("utf-8")
print("Prediction result:", result)



Prediction result: 0.008737483061850071

